# Дерево отрезков

Рассмотрим следующую задачу. Дан массив $a$ из $n$ целых чисел, нужно уметь отвечать на запросы двух типов:

1. Изменить значение в ячейке (т. е. отреагировать на присвоение `a[k] = x`).
2. Вывести сумму элементов $a_i$ на отрезке с $l$ по $r$.

Оба запроса нужно обрабатывать за время $O(n \log n)$.

Чтобы решить задачу, сделаем с исходным массивом следующие манипуляции:

Посчитаем сумму всего массива и где-нибудь запишем. Потом разделим его пополам и посчитаем сумму на половинах. Каждую половину потом разделим пополам ещё раз и так далее, пока не придём к отрезкам длины 1.

Эту последовательность разбиений можно представить в виде дерева.

<картинка>

Корень этого дерева — отрезок $[0, n-1]$, а каждая вершина имеет ровно двух сыновей, кроме вершин-листьев, у которых отрезок имеет длину 1. Отсюда и название — «дерево отрезков».

Заметим, что оно содержит менее $2n$ вершин: первый уровень дерева отрезков содержит одну вершину (корень), второй уровень — в худшем случае две вершины, на третьем уровне в худшем случае будет четыре вершины, и так далее, пока число вершин не достигнет n. Таким образом, число вершин в худшем случае оценивается суммой $n + n/2 + n/4 + n/8 + \ldots + 1 < 2n$.

При n, отличных от степеней двойки, не все уровни дерева отрезков будут полностью заполнены. Например, при n=3 левый сын корня есть отрезок $[0, 1]$, имеющий двух потомков, в то время как правый сын корня — отрезок [2, 2], являющийся листом. Это надо иметь в виду при реализации.

Высота дерева отрезков есть $O(\log n)$: длина отрезка в корне дерева равна $n$, а при переходе на один уровень вниз длина отрезков уменьшается примерно вдвое.

Строить его можно рекурсивной функцией:
* Если вершина является листом, взять в качестве суммы значение соответствующей ячейки.
* Если вершина является отрезком, разделить его на два и в качестве суммы взять сумму его детей.

# Ок, зачем оно надо?

Опишем теперь, как с его помощью решить задачу.

**Запрос обновления**. Нам нужно перестроить дерево таким образом, чтобы оно соответствовало новому значению $a[k] = x$.

Давайте изменим все вершины, в которых участвует $k$-тый элемент. Их будет $O(\log n)$ — по одной с каждого уровня.

Это можно реализовать как рекурсивную функцию: ей передаётся текущая вершина дерева отрезков, и эта функция выполняет рекурсивный вызов от одного из двух своих сыновей (от того, который содержит $k$-ый элемент в своём отрезке), а после этого — пересчитывает значение суммы в текущей вершине точно таким же образом, как мы это делали при построении дерева отрезков.

**Запрос суммы**. Мы знаем, что во всех вершинах лежат корректные значения.

Сделаем тут тоже рекурсивную функцию, рассмотрев три случая

* Если отрезок запроса лежит целиком в 
* Если отрезок запроса лежить целиком вне, то вернуть 0.
* Иначе разделиться рекурсивно на 2 и вернуть сумму.

Чтобы разобраться, почему это работает за $O(\log n)$, нужно оценить количество «интересных» отрезков: тех, которые порождают что-то в рекурсии. 

# Реализация

<Ликбез про структуры и указатели>.

In [ ]:
struct segtree {
    int lb, rb;
    int sum = 0;
    segtree *l, *r;
    segtree (int _lb, int _rb) {
        lb = _lb, rb = _rb;
        if (lb != rb) {
            int t = (lb + rb) / 2;
            l = new segtree(lb, t);
            r = new segtree(t+1, rb);
        }
    }
    void add (int k, int x) {
        sum += x;
        if (lb != rb) {
            int t = (lb + rb) / 2;
            if (k <= t) l->add(k, x);
            else r->add(k, x);
        }
    }
    int get_sum (int lq, int rq) {
        if (lb >= lq && rb <= rq) return sum;
        if (lb > rq || rb < lq) return 0;
        return l->get_sum(lq, rq) + r->get_sum(lq, rq);
    }
};

# Другие реализации

Эта реализация проста и легко расширяема, но весьма медленная.

**На массивах**. Можно ввести несложную нумерацию вершин, позволяющую при спуске в ребёнка пересчитывать его номер. Это позволит не хранить границы текущего отрезка. Подробнее у [емакса](http://e-maxx.ru/algo/segment_tree).

**«ДО снизу»**. Можно вообще делать все операции итеративно. Так получится раз в 7 быстрее, но код писать намного труднее. Подробнее смотрите в одном посте с [codeforces](https://codeforces.com/blog/entry/18051).

Также можно заменить отрезки на полуинтервалы — так кода будет меньше. Это утверждение относится не только к ДО.

# Отложенные операции

Пусть у нас теперь есть операции «прибавить на отрезке».

Идея в том, чтобы выполнять операции максимально лениво — только тогда, когда их результаты нам понадобятся. В случае с прибавлением на отрезке, можно создать отдельную переменную — сколько нужно прибавить к этому отрезку и ко всем его детям, чтобы значения в них получились корректными.

Можно создать вспомогательную функцию `push`, которая будет гарантировать, что в текущей вершине все значения корректны, и передавать сверху обновления детям, если надо.

# Динамическое построение

А что, если у нас все индексы лежать не от в пределах $10^9$, а, например, $10^9$. Все асимптотики нас по прежнему устраивают ($\log_2 10^6 \approx 20$, $\log_2 10^9 \approx 30$), кроме этапа построения.

Можно решить эту проблему так: можно не строить дерево целиком изначально, а, подобно отложенным операциям, создавать вершины тогда, когда в них нужно будет записать что-то не дефолтное.

Реализовать это можно так же, как и с push-ем: в начале всех методов проверять, что дети-вершины созданы.

# Персистентность